In [108]:
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [109]:
with open(r'C:\Users\nelso\OneDrive\Machine_Learning\Project\tweets.txt', encoding = 'utf8') as f:
    tweets = f.readlines()

del tweets[0]

In [110]:
with open(r'C:\Users\nelso\OneDrive\Machine_Learning\Project\tweets.txt', encoding = 'utf8') as f:
    content=list(set(f.read()))

In [111]:
vocab_size = len(content)
ix_to_char = {ix:char for ix, char in enumerate(content)}
char_to_ix = {char:ix for ix, char in enumerate(content)}

A few notes on building the neural network
- Sequence length should vary for each tweet. Should be length of tweet.
- Need to create an \<EOS> marker
- Embedding happens when putting input tweet into x tensor
- From embedding, x should be passed through LSTM
- Then need a hidden layer which should maybe spit out something the size of vocab?
- Use a softmax to get the probability
- NLLL to get the loss
- Backprop
- Run this through 2 for loops: for epoch and for tweet

In [132]:
epochs = 2
hidden_dim = 128
embed_size = 10 # This is going to probably get up to 128 or 256

losses = []
loss_function = nn.NLLLoss()
model = nn.LSTM(embed_size, hidden_dim)
optimizer = optim.SGD(model.parameters(), lr=0.001)


for epoch in range(epochs):
    total_loss = 0
    for tweet in tweets:
        #Create the index for our tweet and our target
        tweet_x = tweet[:-1]
        context_size = len(tweet_x)
        tweet_id = torch.tensor([char_to_ix[c] for c in tweet_x], dtype=torch.long)
        target_tweet = tweet[1:]
        target_ix = torch.tensor([char_to_ix[c] for c in target_tweet], dtype=torch.long) 
        
        #Reset gradient for each iteration
        model.zero_grad()
        
        #Embed characters and forward pass through Relu and Softmax
        embed = nn.Embedding(vocab_size, embed_size)
        embeds = embed(tweet_id).view((1, -1))
        lin_1 = nn.Linear(context_size * embed_size, hidden_dim)
        out = F.relu(lin_1(embeds))
        lin_2 = nn.Linear(hidden_dim, vocab_size)
        out = lin_2(out)
        log_probs = F.log_softmax(out, dim=1)
        
        #Compute the loss function
        loss = loss_function(log_probs, target_ix)
        
        #Backprop and Update Gradient
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    losses.append(total_loss)
print (losses)
        
        
        
        
        

ValueError: Expected input batch_size (1) to match target batch_size (177).